# 잠재다리클레 할당 시각화
https://wikidocs.net/30708 참고


# Stopword 설정
stopword설정. 각 파일마다 stopword를 다르게 설정해두었다. 기본적인 stopword는 한국어 기본 stopword는 https://www.ranks.nl/stopwords/korean에서 참고하였고
그 외에는 토큰화 후 데이터를 살펴보며 필요 없는 단어들을 직접 추가해주었다. 
삭제된 단어의 예시
1. 필요없는 숫자들의 나열
2. 필요 없는 조사
3. 토큰화시 온전한 단어로 인지하지 못하는 단어들을 삭제  ex)시립미술관을 시립 + 미술관으로 분리함



In [1]:
import pandas as pd

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import string
import json


import numpy as np 
import pandas as pd 
from time import time
import re
import string
import os
import collections

from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

In [2]:
from konlpy.tag import Okt, Mecab
okt = Okt()
mecab = Mecab()

In [21]:
df = pd.read_csv("/home/aiffel-dj57/spacing/art_culture_complex_spacing.csv")

/home/aiffel-dj57/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df['comment'] = df.apply(lambda row: mecab.morphs(row['comment']), axis=1)

In [5]:
df['comment'][:10]

0                  [대전, 시민, 을, 위한, 최대, 규모, 의, 공연, 시설물]
1    [코로나, 때문, 에, 너무, 오랜만, 에, 가, 본, 예술, 의, 전당, 이, 였...
2               [시설, 이, 깨끗, 하, 고, 음향, 이, 좋, 은, 곳, 입니다]
3    [굿, 클래식, 코, 로, 나인데, 도, 연주자, 들, 안, 전하, 게, 무대, 에...
4                                                  [굿]
5        [쾌적, 하, 고, 깨끗, 하, 고, 넓, 고, 아주, 좋, 은, 장소, 입니다]
6                                             [깔끔, 해요]
7                             [공연, 봤, 어요, 크, 고, 좋, 네유]
8    [넓직, 한, 공간, 그냥, 밖, 에, 있, 어도, 휴식, 공간, 이, 되, 고, ...
9                                     [풍류, 시, 노래, 콘서트]
Name: comment, dtype: object

## Stopword 설정

In [6]:
from nltk.corpus import stopwords
stop_words = "500 공간 20 해서 보다 다는 아 휴 면서 어 으면 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 고 하지마 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만  대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓 의 함 도 고 로 라는 이 가 어요 합니다 게 다 은 기 어서 곳 는 서요 ㅜㅜ 네요 아요 입니다 세요 는데 해소 50 으며 아서 대전 정말 매우 너무 아주 그냥 ㅎㅎ 인데 흐르 많이 근처 10 주변 위한 욕할"
stop_words = stop_words.split(' ')
print(stop_words[:3])

['500', '공간', '20']


In [7]:
#stopword제거
df['comment'] = df['comment'].apply(lambda x: [word for word in x if word not in (stop_words)])

In [8]:
df['comment'][:10]

0                                [시민, 최대, 규모, 공연, 시설물]
1    [코로나, 오랜만, 본, 예술, 전당, 였, 공연, 았, 직원, 분, 친절, 했, ...
2                                         [시설, 깨끗, 음향]
3    [굿, 클래식, 코, 나인데, 연주자, 전하, 무대, 올려, 셔서, 기획, 님, 께...
4                                                  [굿]
5                                      [쾌적, 깨끗, 넓, 장소]
6                                                 [깔끔]
7                                          [공연, 봤, 네유]
8                   [넓직, 밖, 어도, 휴식, 실내, 늘, 품격, 공연, 한다]
9                                     [풍류, 시, 노래, 콘서트]
Name: comment, dtype: object

In [9]:
tokenized_doc = df['comment'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc[:5])

0                                [시민, 최대, 규모, 공연, 시설물]
1    [코로나, 오랜만, 예술, 전당, 공연, 직원, 친절, 구요, 공연, 최대한, 려구...
2                                         [시설, 깨끗, 음향]
3    [클래식, 나인데, 연주자, 전하, 무대, 올려, 셔서, 기획, 고맙, 다고, 얘기...
4                                                   []
Name: comment, dtype: object


In [10]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['comment'] = detokenized_doc 

tokenized_doc.to_csv("token_artcomplex.csv", mode='w')

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features= 300) # max_features = 상위 300개의 단어를 보존 
X = vectorizer.fit_transform(df['comment'])
X.shape # TF-IDF 행렬의 크기 확인

(823, 300)

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top=lda_model.fit_transform(X)

In [14]:
terms = vectorizer.get_feature_names() # 단어 집합.300개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('전시', 19.58), ('저렴', 12.92), ('장료', 5.82), ('추천', 4.18), ('가족', 4.14)]
Topic 2: [('공연', 12.6), ('장소', 6.7), ('멋진', 6.55), ('다양', 5.21), ('주차', 4.19)]
Topic 3: [('여유', 5.4), ('최고', 4.66), ('만족', 2.69), ('오래', 2.27), ('행복', 2.21)]
Topic 4: [('가족', 6.49), ('깔끔', 5.58), ('재미있', 4.01), ('주차장', 2.81), ('처음', 2.67)]
Topic 5: [('유일', 4.44), ('로운', 4.26), ('흥미', 3.94), ('시원', 3.8), ('전시회', 3.23)]
Topic 6: [('그림', 3.58), ('기획', 3.47), ('무대', 2.42), ('방문', 1.61), ('작가', 1.55)]
Topic 7: [('미술', 5.82), ('경관', 2.69), ('감상', 2.67), ('작품', 2.45), ('전시관', 2.11)]
Topic 8: [('미술관', 17.07), ('작품', 11.63), ('감상', 4.9), ('입장료', 4.39), ('데이트', 3.53)]
Topic 9: [('재밌', 3.68), ('분위기', 3.54), ('쾌적', 3.36), ('야외', 3.28), ('사진', 3.23)]
Topic 10: [('관람', 14.03), ('예술', 11.65), ('문화', 11.26), ('미술관', 9.9), ('공연', 8.87)]


In [15]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) 
#copora dictionary를 활용하여 tokenized_doc을 dictionary화 한다.

[(0, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [16]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(0, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(16, 1), (17, 1), (18, 1)], [(19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1)], [], [(16, 1), (32, 1), (33, 1)], [(34, 1)], [(0, 1), (35, 1)], [(0, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1)], [(42, 1), (43, 1), (44, 1)], [(29, 1), (45, 1)], [(0, 1)], [(0, 1), (9, 1), (15, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1)], [(0, 1), (63, 1), (64, 1), (65, 1), (66, 1)], [(23, 1), (67, 1)], [(48, 1), (60, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1)], [(76, 1)], [(0, 1), (48, 2), (77, 1), (78, 1), (79, 1)], [], [(71, 1), (80, 1), (81, 1), (82, 1)], [(83, 1), (84, 1)], [(9, 1), (11, 1)], [(12, 1), (14, 1), (65, 1), (85, 1)], [(86,

In [17]:
print(dictionary[66])
#66번에 매핑된 단어를 뽑아본다.

연광철


In [18]:
import gensim
NUM_TOPICS = 4 #20개의 토픽, k=20 #여기서 토픽은 단어 군집의 수이다.
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)
#ldamodel    

(0, '0.072*"공연" + 0.019*"예술" + 0.016*"장소" + 0.013*"관람"')
(1, '0.052*"미술관" + 0.022*"산책" + 0.019*"수목원" + 0.013*"한밭"')
(2, '0.042*"전시" + 0.034*"작품" + 0.020*"문화" + 0.016*"미술관"')
(3, '0.014*"주차장" + 0.010*"공원" + 0.010*"미술관" + 0.010*"친절"')


In [19]:
import pyLDAvis.gensim #lda 시각화 툴. 

In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/home/aiffel-dj57/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
